# Relatorio Técnico Referente a Matéria de Técnicas de Pesquisa Experimental
  
**Tema:** Analise da Variação de Parâmetros na Impressão 3D de Filamentos Poliméricos  
**Profº:** Pedro Bastos Costa  
**Universidade Federal de Minas Gerais - UFMG**

**Membros:**
* Guilherme de Paula Rúbio  
* Matheus Ungaretti Borges  
* Daniel Affonso Vasconcelos  

## *Importação de Bibliotecas*

In [8]:
import numpy as np 
import pandas as pd 
import scipy.stats as st 
import scipy.linalg as ln
import matplotlib.pyplot as plt 

from math import sqrt

%matplotlib inline

ModuleNotFoundError: No module named 'scipy'

## ***Fuções Auxiliares Criadas***  

* **Função**: Teste de hipótese bicaudal, distribuição F

In [ ]:
def F_bicaudal_hypoteses (F :float, alpha :float, dn :float, dd :float):
    '''
        Função de teste de hipótese de um teste bicaudal do tipo F
            Entrada:
               * F: Valor calculado de F_0;
               * alpha: nivel de confiança total;
               * dn: graus de liberdade do numerador;
               * dd: graus de liberdade do denominado.

            Saídas:
               * Impressão das Respostas.

            Retorno: Nenhum
    '''
    min = st.f.ppf(alpha/2, dn, dd)         # Limite Superior
    max = st.f.ppf(1-(alpha/2), dn, dd)     # Limite Inferior

    print('O valor da estatística cálculada F0 é: {:.4f}'.format(F))
    print('O limites da distribuição para {} de {:.2f} é: [{:.4f}:{:.4f}]'.format('\u0251', alpha, min, max))
    print('Portanto,',end=' ')

    if (F<min) or (F>max):
        print('\033[33mREJEITA-SE\033[m a hipótese nula!')
        print('p-valor: {:.4e}'.format( st.f.cdf(F, dn, dd) if (F<min) else (1-st.f.cdf(F, dn, dd)) ))
    else:
        print('\033[31mFALHA EM REJEITAR\033[m a hipótese nula!')

   * **Função:** Cálculo do erro padrão de uma regressão linear múltipla

In [ ]:
def lin_mult_regress_error (C0 :np.array, var2 :float):
    '''
    Função de calculo de erro padrão de cada constantes:
        Entradas:
            * C0 = Matriz de covariância da regressão
            * var2 = variância do erro padrão
        Saida/Retorno:
            * se = array com os erros padrões
    '''
    se = []

    for i in range(len(C0)):
        se.append(sqrt(var2*C0[i,i]))
    
    return se

   * **Função:** Teste de hipótese dos coeficientes:  

In [ ]:
def lin_mult_coef_hipo (B :np.array, se :np.array, Bj0 :float, alpha :float, n :int):
    '''
        Função de teste de hipótese dos coeficientes de uma regressão linear
            Entradas:
                * B: vetor com os coefcientes calculados
                * se: vetor com o erro padrão dos coeficientes
                * Bjo: hipótese nula
                * alpha: nível de confiança
                * n: n° de parâmetros da amostra
            
            Saídas:
                * Impressão dos resultados na tela
            
            Retorno: 
                *   Nenhum
    '''
    df = n - (len(B)-1)

    for i in range(len(B)):
        B0 = (B[i]-Bj0)/se[i]

        min = st.t.ppf(alpha/2,df)
        max = st.t.ppf(1-(alpha/2),df)

        print('{}'.format('=*='*20))
        print('{:^60}'.format('Cálculo de {}{}'.format('\u03B2',i)))

        print('O valor de T0 é: {:.4f}'.format(B0))
        print('O valor dos limites da região crítica é: [{:.4f}:{:.4f}]'.format(min, max))
        print('Assim, ', end='')

        if (B0 < min) or (B0 > max):
            print('\033[33mREJEITA-SE\033[m a hipótese nula!')
            print('O valor de p é: {:.4e}'.format(st.t.cdf(B0,df) if B0 < min else 1-st.t.cdf(B0,df)))
        else:
            print('\033[31mFALHA-SE EM REJEITAR\033[m a hipótese nula!')
        
        print('{}'.format('=*='*20),'\n')

## *Importação dos dados*
Todos os dados utilizados nessa análise estão disponibilizados na plataforma Kaggle. Caso queria acessar a fonte dos dados utilizados basta [clicar aqui.](https://www.kaggle.com/afumetto/3dprinter?select=data.csv)  

### Contexto  
Os dados utilizados são oriundos de uma pesquisa desenvolvida pelo Departamento de Engenharia Mecânica da TR/Selcuk University.  
Essa pesquisa visa avaliar quais os parâmetros de impressão interferem na qualidade de impressão de peças, precisão e rigidez.  
Nos dados apresentados existem nove parâmetros de configuração (entradas), e de três parâmetros de saída.  

### Materiais e Métodos  
* **Impressora:** Ultimaker S5 3-D 
* **Teste de materiais e resistência:** Sincotec GMBH, capacidade de tração - 20kN.

### Conteúdo dos dados
#### Entradas - Parâmetros de configuração:  
* Altura de camada (*Layer Height*) \[mm]  
* Espessura de casca (*Wall Thickness*) \[mm]  
* Densidade de preenchimento (*Infill Density*) \[%]  
* Padrão de preenchimento (*Infill Pattern*)  
* Temperatura do bico de extrusão (*Nozzle Temperature*) \[°C]  
* Temperatura da mesa de impressão (*Bed Temperature*) \[°C]  
* Velocidade de impressão (*Print Speed*) \[mm/s]  
* Material (*Material*)  
* Velocidade do ventilador da extrusora (*Fan Speed*) \[%]  

#### Saídas  
* Rugosidade (*Roughness*) \[µm]
* Tensão de ruptura (*Ultimate Tension Strenght*) \[MPa]
* Alongação (*Elongation*) \[%]  

Abaixo importamos os dados para a análise.  

In [ ]:
dados = pd.read_csv('datasets\data_3D_print.csv')   # Importa o DataFrame
dados.head()     # Apresenta as 5 primeiras linhas do DataFrame

## *Análise das Saídas*  
Como temos três saidas iremos analisá-las separadamente.  

### Saida Rugosidade:  

#### ***Influência do material***  
Umas das primeiras hipóteses que desejamos testar do ponto de vista da rugosidade, é se o material afeta estatisticamente seu valor, para isso separaremos os valores de saída de rugosidade por material para realizar uma análise de experimento com um único fator.

In [ ]:
dados_roughness_material = dados[['material','roughness']]
dados_roughness_material.head()

Com os dados separados podemos observar que nosso problema é dividido em dois materiais, ou seja, dois níveis (PLA e ABS). Primeiramente vamos analisar quantas observações de cada material nós temos.  

In [ ]:
dados_roughness_material['material'].value_counts()

Como podemos observar, o número de observações de cada material é igual.
***
#### Teoria 📖:  
Supomos que nosso valor de saída, rugosidade, varia de acordo com o material utilizado, o que significaria que ela teria o seguinte comportamento:  
$$
\mathsf{
    Y_{ij}= \mu + \tau_{i} + \epsilon_{ij}
}
\displaystyle
    \begin{cases}{}
        \mathsf{i=0, 1}\\
        \mathsf{j=0, 1, 2, 3, \dots, 25}
    \end{cases}
$$  
Onde:
   * $\mu$: média global;  
   * $\tau_{i}$: fator de alteração referente ao material utilizado;
   * $\epsilon_{ij}$: erro aleatorio da medição  
***  
Para garantirmos que a escolha do material realmente interfere na rugosidade medida nas nossas peças, temos que garantir que o valor de $\tau$ de cada material seja diferente de zero. Logo, temos que fazer um teste assumindo as seguintes hipóteses:  
$$
\left\{
    \begin{matrix}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{H_1}: \tau_{0}\neq \tau_{1}\neq 0}
    \end{matrix}
\right.
$$  
Para isso faremos um teste ANOVA, que se baseia em assumir que o modelo é descrito por uma estatística F, assim:  
$$
\mathsf{
    F_0=\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i.} - \bar{y}_{..}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i.}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  

Para iniciarmos os cálculos vamos primeiramente calcular as médias e armazena-las

In [ ]:
mean_roug_mat = dados_roughness_material.groupby('material').mean()   # media da rugosidade por material
mean_roug_mat

In [ ]:
mean_roug_mat_gen = dados_roughness_material['roughness'].mean()  # Média Geral 
mean_roug_mat_gen

Calculando o $SQ_{tratamentos}$:

In [ ]:
n = (dados_roughness_material['material'].value_counts()).loc['abs']
SQ_trat = n * sum((mean_roug_mat['roughness']-mean_roug_mat_gen)**2)
SQ_trat

Calculando $SQ_E$

In [ ]:
SQ_abs = sum((dados_roughness_material[dados_roughness_material['material']=='abs']['roughness']-float(mean_roug_mat.loc['abs']))**2)
SQ_pla = sum((dados_roughness_material[dados_roughness_material['material']=='pla']['roughness']-float(mean_roug_mat.loc['pla']))**2)

SQ_e = SQ_abs+SQ_pla
SQ_e

Com os erros quadráticos calculados, podemos fazer o teste de hipótese:  

In [ ]:
a = len(mean_roug_mat)

MQ_trat = SQ_trat/(a-1)
MQe = SQ_e/(a*(n-1))

F0 = MQ_trat/MQe

F_bicaudal_hypoteses(F0, 0.05, a-1, a*(n-1))

Como podemos observar para um $\alpha$ de 0,05 FALHAMOS EM REJEITAR a hipótese nula, logo o material não afeta estatísticamente o valor de rugosidade.   
Além do teste ANOVA utilizaremos então o método MDS para avaliar se as médias encontradas têm diferença significativa no valor da rugosidade.  
A medida da significatividade entre médias é dada pela relação:  
$$
\mathsf{
    |\bar{y}_{i.}-\bar{y}_{j.}|> MDS
}
$$  
$$
\mathsf{
    |\bar{y}_{i.}-\bar{y}_{j.}|> \mathit{t}_{\frac{\alpha}{2};a \cdot(n-1)} \cdot \sqrt{\dfrac{2\cdot MQ_E}{n}}
}
$$  
Assim:  

In [ ]:
MDS = st.t.ppf(1-0.025,a*(n-1)) * sqrt((2*MQe)/n)

print('{:^10}|{:^10}'.format('abs-pla','MDS'))
print('{:^1}{:.4f}{:^2}|{:^1}{:.4f}{:^2}'.format('',float(mean_roug_mat.loc['abs'] - mean_roug_mat.loc['pla']),'','', MDS,''))

Analisando a mínima diferença significativa (MDS), vemos que realmente a mudança do material não afeta estatisticamente o valor da rugosidade.  
Essa conclusão era esperada, uma vez que, como a impressão ocorre por camadas, a rugosidade deve ser afetada principalmente por parâmetros que determinam as caracteristicas dessas camadas, mais do que pelo material em si.  

### ***Análise do preenchimento interno***  
Com a influência do material sendo descartada agora avaliaremos se o preenchimento interno da peça influencia na rugosidade da peça.  
Primeiramente vamos separar os dados de preenchimento interno e rugosidade.   

In [ ]:
pattern_roughness = dados[['infill_pattern','roughness']]
pattern_roughness.head()

In [ ]:
pattern_roughness['infill_pattern'].value_counts()

Separados os dados vamos então realizar um teste ANOVA seguindo as hipóteses:  
$$
\left\{
    \begin{matrix}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{H_1}: \tau_{0}\neq \tau_{1}\neq 0}
    \end{matrix}
\right.
$$  
Para isso assumimos que o modelo assume uma estatística F, assim:  
$$
\mathsf{
    F_0=\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i.} - \bar{y}_{..}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i.}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  
Assim calculamos $F_0$ e fazemos o teste de hipóteses:  

In [ ]:
patt_rough_mean = pattern_roughness.groupby('infill_pattern').mean()
patt_rough_mean

In [ ]:
patt_rough_gen = pattern_roughness.mean()
patt_rough_gen

In [ ]:
n_ptt = (pattern_roughness['infill_pattern'].value_counts()).loc['grid']
SQ_tra_ptt = n_ptt * sum( (patt_rough_mean['roughness'] - float(patt_rough_gen))**2 )
SQ_tra_ptt

In [ ]:
SQ_e_grid = sum((pattern_roughness[pattern_roughness['infill_pattern']=='grid']['roughness'] - float(patt_rough_mean.loc['grid']))**2)
SQ_e_hone = sum((pattern_roughness[pattern_roughness['infill_pattern']=='honeycomb']['roughness'] - float(patt_rough_mean.loc['honeycomb']))**2)

SQ_e_ptt = SQ_e_grid + SQ_e_hone
SQ_e_ptt

In [ ]:
a_ptt = len(patt_rough_mean)

MQ_trat_ptt = SQ_tra_ptt/(a-1)
MQe_ptt = SQ_e_ptt/(a*(n-1))

F0_ptt = MQ_trat_ptt/MQe_ptt

F_bicaudal_hypoteses(F0, 0.05, a-1, a*(n-1))

#### ***Modelo de Regressão Linear Multipla***  
Com os dados restantes vamos fazer um modelo de regressão multipla com 7 variáveis $\mathsf{\left(x_1, x_2, x_3, x_4, x_5, x_6\right)}$, sendo cada uma dessas uma entrada restante, e possuem 8 coeficientes $\mathsf{\left(\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}_2, \hat{\beta}_3, \hat{\beta}_4, \hat{\beta}_5, \hat{\beta}_6\right)}$.  
Primeiramente vamos separar os dados com relação com a rugosidade:

***Obs.:*** Analisando os dados de velocidade do *fan* e da temperatura do bico, podemos observar que um era escrito em função do outro, assim optamos por excluir a coluna desses dados de nossa análise.  

In [ ]:
data_roughness = dados.drop(['infill_pattern', 'material', 'tension_strenght', 'elongation'], axis=1)
data_roughness.head()

***
#### Teoria 📖  
Para determinarmos os coeficientes usaremos a seguinte relação: 
$$
\mathsf{
    \hat{\beta} = \left( X' \cdot X \right)^{-1} \cdot X' \cdot y
}
$$  
Onde:  
$$
\mathsf{y} = \left[
    \begin{matrix}
        \mathsf{y_1}\\ \mathsf{y_2}\\ \vdots \\ \mathsf{y_n}
    \end{matrix}
\right];

\mathsf{X} = \left[
    \begin{matrix}
        1 & \mathsf{x_{11}} & \mathsf{x_{12}} & \cdots & \mathsf{x_{1k}}\\
        1 & \mathsf{x_{21}} & \mathsf{x_{22}} & \cdots & \mathsf{x_{2k}}\\
        \vdots & \vdots & \vdots & \vdots & \vdots\\
        1 & \mathsf{x_{n1}} & \mathsf{x_{n2}} & \cdots & \mathsf{x_{nk}}
    \end{matrix}
\right]; 

\mathsf{\hat{\beta}} = \left[
    \begin{matrix}
        \mathsf{\beta_0}\\ \mathsf{\beta_1}\\ \vdots \\ \mathsf{\beta_0}
    \end{matrix}
\right]
$$  

Desta equação temos que a matriz de covariância é dada por:  
$$
\mathsf{
    C = \left(X' \cdot X \right)^{-1}
}
$$  
***
Agora vamos preencher a matriz de covariância com nossos dados:  

In [ ]:
n = len(data_roughness)
X_roug = np.array(pd.concat([pd.Series(np.ones(len(data_roughness))),data_roughness.iloc[:,:6]],axis=1))
Y_roug = np.array(data_roughness['roughness'])

C = np.linalg.inv((X_roug.T).dot(X_roug))
print(C)

Com a matriz de covariância podemos calcular os coeficientes da regressão:  

In [ ]:
B_rough = (C.dot(X_roug.T)).dot(Y_roug)

for i in range(len(B_rough)):
    print('{}{}: {:.4f}'.format('\u03B2', i, B_rough[i]))

Assim o modelo de múltiplas variáveis resultante é:  
$$
\mathsf{
    \hat{y}_{rugosidade} = -529,1807 + 1262,8094 \cdot x_1 + 1,6442 \cdot x_2 + 0,1694 \cdot x_3 + 2,3852 \cdot x_4 - 0,2978 \cdot x_5 + 0,6373 \cdot x_6
}
$$  

#### Teste para significância  da Regressão  
Para avaliar se a nossa regressão é significativa assumimos as hipóteses:  
$$
\left\{
    \begin{array}{l}
        \mathsf{\mathit{H_0:} \hat{\beta}_i= 0}\\
        \mathsf{\mathit{H_1: } \hat{\beta}_j \not= 0}
    \end{array}
\right.
$$  
Com a estatística:  
$$
\mathsf{
    F_0 = \dfrac{\dfrac{SQ_R}{k}}{\dfrac{SQ_E}{n-p}} 
}
$$  
Onde:
$$
\begin{array}{ccc}
    \mathsf{SQ_R = \displaystyle\sum_{i=1}^{n=50} \left(\hat{y}_i - \bar{y}\right)^2} & \mathsf{e} & \mathsf{k = p-1}
\end{array}
$$  
$$
\mathsf{
    SQ_E = \sum_{i=1}^{n=50}{\left(y_i - \hat{y}\right)^2}
}
$$

In [ ]:
Y_hat_roug = B_rough[0] + B_rough[1]*data_roughness['layer_height'] + B_rough[2]*data_roughness['wall_thickness'] + B_rough[3]*data_roughness['infill_density'] + B_rough[4]*data_roughness['nozzle_temperature'] + B_rough[5]*data_roughness['bed_temperature'] + B_rough[6]*data_roughness['print_speed']

SQe_roug = sum( (Y_roug-Y_hat_roug)**2 )

n_roug = len(Y_roug)
p_roug = len(B_rough)-1
k_roug =  p_roug-1

SQr_roug = sum( (Y_hat_roug - data_roughness['roughness'].mean())**2 )

F0_roug = (SQr_roug/k_roug)/(SQe_roug/(n_roug-p_roug))

F_bicaudal_hypoteses(F0_roug, 0.05, k_roug, n_roug-p_roug)

Como podemos ver, falhamos em rejeitar a hipótese nula. Isso significa que ao menos uma constante tem influência estatística no valor de rugosidade.

Agora basta determinarmos quais das variáveis interferem realmente na rugosidade. Para isso consideraremos as seguintes hipóteses para o teste de coeficientes:  
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0:~}\hat{\beta}_j = 0}\\
        \mathsf{\mathit{H_1:~}\hat{\beta}_j \not=0}
    \end{array}
\right.
$$  
Porém agora utilizaremos a seguinte estatística:  
$$
\mathsf{
    T_0 = \dfrac{\hat{\beta}j - \beta_{j_0}}{\sqrt{\sigma^2 \cdot C_{jj}}} = \dfrac{\hat{\beta}j - \beta_{j_0}}{se(\hat{\beta}_j)}
}
$$ 

Onde: 
$$
\mathsf{
    \hat{\sigma}^2 = \dfrac{SQ_E}{n-p} = \dfrac{\displaystyle \sum_{i=1}^{n=50}{\left(y_i - \hat{y}\right)^2}}{50-7}
}
$$  
Calcularemos agora a variância do erro e o erro padrão de cada coeficiente.  

In [ ]:
var = SQe_roug/(n_roug-p_roug)

seB_roug = lin_mult_regress_error(C, var)
seB_roug

Assim então testaremos cada coeficiente:  

In [ ]:
lin_mult_coef_hipo(B_rough,seB_roug,0,0.05,n_roug)

In [ ]:
data_roughness.columns

Analisando os coeficientes podemos observar que de acordo com o nosso modelo os parâmetros que mais influenciam na rugosidade são:  
   * Altura de Camada  
   * Temperatura do bico  
   * Velocidade de impressão  
Assim reescrevemos nosso modelo sendo:  
$$
\mathsf{
    R = -529,1807 + 1262,8094 \cdot lh + 2,3852 \cdot T + 0,6373 \cdot v
}
$$

Onde:  
   * $\mathsf{R}$ = rugosidade  
   * $\mathsf{lh}$ = altura de camada  
   * $\mathsf{T}$ = temperatura do bico  
   * $\mathsf{v}$ = velocidade de impressão

#### ***Coeficiente de determinação $\mathsf{R^2}$***  
Por fim vamos analisar o quão ajustado nosso modelo ficou para o problema, para isso iremos calcular o coeficente de determinação.  
***
#### **Teoria** 📖  
O coeficente de determinação indica o quanto o tipo de curva encontrado descreve o comportamento das variáveis, e é definido por:  
$$
\mathsf{
   R^2 = \dfrac{SQ_R}{SQ_T} = \dfrac{SQ_R}{SQ_E + SQ_R}
}
$$  
Porém em regressões múltiplas é aconselhavel se utilizar esse valor ajustado, devido ao número de variáveis, assim temos que:  
$$
\mathsf{
   R_{ajustado}^2 = 1 - \dfrac{\left(\dfrac{SQ_E}{n-p}\right)}{\left(\dfrac{SQ_T}{n-1}\right)}
}
$$
*** 
Calculado o coeficiente temos:  

In [ ]:
RR_roug = SQr_roug/(SQe_roug+SQr_roug)
RR_roug_adj = 1 - ( (SQe_roug/(n_roug-p_roug))/((SQr_roug+SQe_roug)/(n_roug-1)) )

print('R²: {:.4f}'.format(RR_roug))
print('R²_ajustado: {:.4f}'.format(RR_roug_adj))

### Saída: Tensão de ruptura 

O mesmo procedimento adotado para testar a rugosidade será novamente utilizado para análise da tensão de ruptura.  
#### *Influência do Material*  
Vamos analisar se em uma peça impressa as resistências de cada material (Ácido Polilático e Acrilonitrila Butadieno Estireno) realmente influenciam na tensão de ruptura da peça final, ou se a diferença entre a resistência de cada um não é o principal fator no resultado final. Pra isso faremos uma análise de experimento dos dados de tensão de ruptura.  

Primeiramente separaremos os dados de material e tensão de ruptura:  

In [ ]:
data_ten_material = dados[['material','tension_strenght']]
data_ten_material.head()

Vamos agora observar quantas observações temos de cada material, ou seja o valor de n.  

In [ ]:
n_ten_mat = data_ten_material['material'].value_counts()
n_ten_mat

Sabendo que o número de observações é igual podemos então fazer o teste ANOVA, com as seguintes hipóteses: 
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{H_1}: \tau_{0}\not=\tau_{1}\not=0}
    \end{array}
\right.
$$  
Assumimos que o modelo assumi uma estatística F, assim:  
$$
\mathsf{
    F_0=\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i\cdot} - \bar{y}_{\cdot\cdot}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i\cdot}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  

Assim temos:  

In [ ]:
mean_ten_mat = data_ten_material.groupby('material').mean()   # media da tensão por material
mean_ten_mat

In [ ]:
mean_ten_mat_gen = data_ten_material['tension_strenght'].mean()  # Média Geral 
mean_ten_mat_gen

Calculando $SQ_{tratamentos}$

In [ ]:
SQtrat_ten = n_ten_mat[0] * sum((mean_ten_mat['tension_strenght']-mean_ten_mat_gen)**2)
SQtrat_ten

Calculando $SQ_E$

In [ ]:
SQ_abs = sum((data_ten_material[data_ten_material['material']=='abs']['tension_strenght']-float(mean_ten_mat.loc['abs']))**2)
SQ_pla = sum((data_ten_material[data_ten_material['material']=='pla']['tension_strenght']-float(mean_ten_mat.loc['pla']))**2)

SQe_ten = SQ_abs+SQ_pla
SQe_ten

Fazendo o teste de hipótese temos:  

In [ ]:
a_ten_mat = len(mean_ten_mat)

MQtrat_ten = SQtrat_ten/(a_ten_mat-1)
MQe_ten = SQe_ten/(a_ten_mat*(n_ten_mat[0]-1))

F0_ten_mat = MQtrat_ten/MQe_ten

F_bicaudal_hypoteses(F0_ten_mat, 0.05, a_ten_mat-1, a_ten_mat*(n_ten_mat[0]-1))

Como falhamos em rejeitar a hipótese de que os materiais PLA e ABS são um fator de influência na resistência final de tração, consideramos que estatisticamente falando, o material não é um fator que influencia a resistência para esses dois materiais testados.  

#### *Influência do Padrão de Preenchimento*
Com a escolha dentre esses dois materiais tendo sido determinada como não influente na tensão de ruptura final, analisaremos agora se o padrão de preenchimento é fator relevante.  
Separaremos os dados que serão utilizados. 


In [ ]:
data_ten_ptt = dados[['infill_pattern', 'tension_strenght']]
data_ten_ptt.head()

Observando a quantidade de observações de cada padrão temos:  

In [ ]:
n_ten_ptt = data_ten_ptt['infill_pattern'].value_counts()
n_ten_ptt

Sabendo que o número de observações é igual podemos então fazer o teste ANOVA, com as seguintes hipóteses: 
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{H_1}: \tau_{0}\not=\tau_{1}\not=0}
    \end{array}
\right.
$$  
Assumimos que o modelo tem o mesmo comportamento de uma estatística F, assim:  
$$
\mathsf{
    F_0=\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i\cdot} - \bar{y}_{\cdot\cdot}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i\cdot}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  

Assim temos:  

In [ ]:
mean_ten_ptt = data_ten_ptt.groupby('infill_pattern').mean()   # media da tensão por padrão
mean_ten_ptt

In [ ]:
mean_ten_ptt_gen = data_ten_ptt['tension_strenght'].mean()  # Média Geral 
mean_ten_ptt_gen

Caculando o $SQ_{tratamentos}$

In [ ]:
SQtrat_ten_ptt = n_ten_ptt[0] * sum((mean_ten_ptt['tension_strenght']-mean_ten_ptt_gen)**2)
SQtrat_ten_ptt

Calculando o $SQ_E$

In [ ]:
SQ_hon = sum((data_ten_ptt[data_ten_ptt['infill_pattern']=='honeycomb']['tension_strenght']-float(mean_ten_ptt.loc['honeycomb']))**2)
SQ_gd = sum((data_ten_ptt[data_ten_ptt['infill_pattern']=='grid']['tension_strenght']-float(mean_ten_ptt.loc['grid']))**2)

SQe_ten_ptt = SQ_abs+SQ_pla
SQe_ten_ptt

Fazendo o teste de hipótese temos:  

In [ ]:
a_ten_ptt = len(mean_ten_ptt)

MQtrat_ptt = SQtrat_ten_ptt/(a_ten_ptt-1)
MQe_ten_ptt = SQe_ten_ptt/(a_ten_ptt*(n_ten_ptt[0]-1))

F0_ten_ptt = MQtrat_ptt/MQe_ten_ptt

F_bicaudal_hypoteses(F0_ten_ptt, 0.05, a_ten_ptt-1, a_ten_ptt*(n_ten_ptt[0]-1))

Como podemos ver o preenchimento interno também não influencia na tensão de ruptura da peças.  

#### ***Regressão Mútlipla***  
Com os dados restantes vamos fazer um modelo de regressão múltipla com 6 variáveis $\mathsf{\left(x_1, x_2, x_3, x_4, x_5, x_6\right)}$, sendo cada uma dessas uma entrada restante, possuindo 7 coeficientes $\mathsf{\left(\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}_2, \hat{\beta}_3, \hat{\beta}_4, \hat{\beta}_5, \hat{\beta}_6\right)}$.  
Assim como no caso para rugosidade desconsideraremos os dados de velocidade do *fan*, por ele ser dependente da temperatura.  
Separando os dados primeriamente:  

In [ ]:
data_ten = dados.drop(['infill_pattern', 'material', 'fan_speed', 'roughness', 'elongation'], axis=1)
data_ten.head()

Montaremos primeriamente a matriz de covariância:  

In [ ]:
X_ten = pd.concat([pd.Series(np.ones(len(data_ten))),data_ten.iloc[:,:6]],axis=1)

C_ten = np.linalg.inv((X_ten.T) @ X_ten)
C_ten

Com a matriz de covariância podemos então encontrar os coeficientes da regressão.  

In [ ]:
Y_ten = data_ten['tension_strenght']

B_ten = (C @ (X_ten.T)) @ Y_ten
for i in range(len(B_ten)):
    print('{}{}: {:.4f}'.format('\u03B2', i, B_ten[i]))

Assim então temos que o modelo de regressão linear é dado por: 
$$
\mathsf{
    \hat{y}_{tensão de ruptura} = 63,4423 + 55,6460 \cdot x_1 + 1,0743 \cdot x_2 + 0,1537 \cdot x_3 -0,3047 \cdot x_4 + 0,0777 \cdot xx_5 - 0,0161 \cdot x_6
}
$$  

#### Teste de Signficância  
Com os coeficientes calculados vamos determinar se a regressão realmente mostra uma relação dos dados seguindo a hipótese:  
$$
\left\{
    \begin{array}{l}
        \mathsf{\mathit{H_0:} \hat{\beta}_i= 0}\\
        \mathsf{\mathit{H_1: } \hat{\beta}_j \not= 0}
    \end{array}
\right.
$$  
Com a estatística:  
$$
\mathsf{
    F_0 = \dfrac{\dfrac{SQ_R}{k}}{\dfrac{SQ_E}{n-p}} 
}
$$  
Onde:
$$
\begin{array}{ccc}
    \mathsf{SQ_R = \displaystyle\sum_{i=1}^{n=50} \left(\hat{y}_i - \bar{y}\right)^2} & \mathsf{e} & \mathsf{k = p-1}
\end{array}
$$  
$$
\mathsf{
    SQ_E = \sum_{i=1}^{n=50}{\left(y_i - \hat{y}\right)^2}
}
$$

In [ ]:
Y_hat_ten = B_ten[0] + B_ten[1]*data_ten['layer_height'] + B_ten[2]*data_ten['wall_thickness'] + B_ten[3]*data_ten['infill_density'] + B_ten[4]*data_ten['nozzle_temperature'] + B_ten[5]*data_ten['bed_temperature'] + B_ten[6]*data_ten['print_speed']

SQe_ten = sum( (Y_ten-Y_hat_ten)**2 )

n_ten = len(Y_ten)
p_ten = len(B_ten)-1
k_ten =  p_ten-1

SQr_ten = sum( (Y_hat_ten - data_ten['tension_strenght'].mean())**2 )

F0_ten = (SQr_ten/k_ten)/(SQe_ten/(n_ten-p_ten))

F_bicaudal_hypoteses(F0_ten, 0.05, k_ten, n_ten-p_ten)

Como podemos ver rejeitamos a hipótese nula, logo ao menos uma das constantes interfere na tensão de ruptura.  

Determinaremos então quais das variáveis interferem realmente na rugosidade. Para isso consideraremos as seguintes hipóteses para o teste de hipóteses dos coeficientes:  
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0:~}\hat{\beta}_j = 0}\\
        \mathsf{\mathit{H_1:~}\hat{\beta}_j \not=0}
    \end{array}
\right.
$$  
Porém agora utilizaremos a seguinte estatística:  
$$
\mathsf{
    T_0 = \dfrac{\hat{\beta}j - \beta_{j_0}}{\sqrt{\sigma^2 \cdot C_{jj}}} = \dfrac{\hat{\beta}j - \beta_{j_0}}{se(\hat{\beta}_j)}
}
$$ 

Onde: 
$$
\mathsf{
    \hat{\sigma}^2 = \dfrac{SQ_E}{n-p} = \dfrac{\displaystyle \sum_{i=1}^{n=50}{\left(y_i - \hat{y}\right)^2}}{50-7}
}
$$  
Primeiramente calcularemos a variância do erro e o erro padrão de cada coeficiente.

In [ ]:
var_ten = SQe_ten/(n_ten-p_ten)

seB_ten = lin_mult_regress_error(C_ten, var_ten)
seB_ten

Assim podemos testar os coeficientes:  

In [ ]:
lin_mult_coef_hipo(B_ten, seB_ten, 0, 0.05, n_ten)

In [ ]:
data_ten.columns

Como podemos observar, os coeficientes relativos a velocidade de impressão e temperatura da mesa não falham em rejeitar a hipótese nula. Logo o modelo final do nosso sistema é:  
$$
\mathsf{
    \sigma_r = 63,4423 + 55,6460 \cdot lh + 1,0743 \cdot wt + 0,1537 \cdot d_i - 0,3047 \cdot T
}
$$  
Onde:  
   * $\mathsf{\sigma_r}$ = tensão de ruptura  
   * $\mathsf{lh}$ = altura de camada  
   * $\mathsf{wt}$ = espessura de camada  
   * $\mathsf{d_i}$ = densidade de preenchimento
   * $\mathsf{T}$ = temperatura do bico  

Por fim calcularemos o coeficiente de determinação:  

In [ ]:

RR_ten = SQr_ten/(SQe_ten+SQr_ten)
RR_ten_adj = 1 - ( (SQe_ten/(n_ten-p_ten))/((SQr_ten+SQe_ten)/(n_ten-1)) )

print('R²: {:.4f}'.format(RR_ten))
print('R²_ajustado: {:.4f}'.format(RR_ten_adj))

### Tensão de ruptura x elongação  

Por fim vamos fazer uma regressão linear simples para enteder a relação "tensão X deformação" de peças impressas. Como vimos anteriormente que o material pouco interfere na tensão de ruptura vamos considerar os dados de apenas um material.  

Utilizando-se a função estatística para regressão linear temos:  

In [ ]:
regr_ten_def = st.linregress(dados['elongation'],dados['tension_strenght'])

print('y = {:.4f} + {:.4f}.x'.format(regr_ten_def.intercept, regr_ten_def.slope))

O coeficiente de determinação da reta é:  

In [ ]:
print('R² = {:.4f}'.format(regr_ten_def.rvalue**2))

O resultado é um ajuste moderado da reta.  
Para finalizar, abaixo encontra-se um gráfico da relação da tensão de ruptura em função do alongamento.

In [ ]:
plt.figure(1, figsize=(14,9))
plt.title('Tensão de Ruptura x Alongamento', fontfamily = 'serif', fontsize = 18, fontweight = 'bold')
plt.grid(True, linewidth = 2, linestyle =':', color='b', alpha=0.5)
plt.plot(dados['elongation'],dados['tension_strenght'], 'o', label='Dados dispersos', color='r')
plt.plot(dados['elongation'],regr_ten_def.intercept+(regr_ten_def.slope*dados['elongation']), 
    label = 'Reta Construida', color='m')
plt.legend(fontsize=12,frameon=True)
plt.xlabel('Alongamento [%]',fontsize=14,fontfamily='serif')
plt.ylabel('Tensão de Ruptura [Mpa]',fontsize=14,fontfamily='serif')
plt.yticks(np.arange(0,40,1.25))
plt.ylim(0,37.5)
plt.xticks(np.arange(0,3.5,0.1),rotation=60)
plt.xlim(0,3.4)